In [1]:
#한글 폰트 설치
!apt-get install -y fonts-nanum > /dev/null
!fc-cache -fv > /dev/null
!rm -rf ~/.cache/matplotlib > /dev/null
#런타임 다시시작

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['axes.unicode_minus'] = False
mpl.rcParams['font.family'] = 'NanumBarunGothic'

In [3]:
#konlpy 설치
!pip install konlpy > /dev/null

In [ ]:
# 기존 chromium 삭제
!sudo apt remove chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


# Install selenium
pip install selenium

In [ ]:
!sudo apt update
!sudo apt install chromium

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm
from urllib.parse import quote

In [36]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [37]:
base_url = 'https://kin.naver.com/search/list.naver?query='
url = base_url + quote('여친선물')
driver.get(url)

In [40]:
answer_list = []
for page in range(1,11):
  print('page', page)
  url = f"{base_url}{quote('여친선물')}&page={page}"
  driver.get(url)    
  time.sleep(3)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  lis = soup.select('ul.basic1 > li')

  for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)
    time.sleep(3)

    count = int(driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text)
    answer_page = (count -1) // 5
    print(count, answer_page)
    for i in range(answer_page):
      driver.find_element(By.ID, 'nextPageButton').click()
      time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    divs = soup.select('div.answer-content__item._contentWrap._answer')
    print(len(divs))
    for div in divs:
      ps = div.select('p.se-text-paragraph')
      for p in ps:
        text = p.get_text()
        if text == " " or text == '\u200b':
          continue
        answer_list.append(text)
    driver.back()
    time.sleep(3)

page 1
8 1
8
8 1
8
4 0
4
10 1
10
12 2
12
1 0
1
5 0
5
6 1
6
3 0
3
1 0
1
page 2
12 2
12
11 2
11
5 0
5
13 2
13
12 2
12
6 1
6
1 0
1
4 0
4
14 2
14
1 0
1
page 3
14 2
14
5 0
5
2 0
2
1 0
1
8 1
8
9 1
9
2 0
2
5 0
5
3 0
3
1 0
1
page 4
3 0
3
1 0
1
5 0
5
4 0
4
4 0
4
9 1
9
3 0
3
18 3
18
1 0
1
2 0
2
page 5
2 0
2
4 0
4
1 0
1
5 0
5
8 1
8
9 1
9
14 2
14
13 2
13
13 2
13
15 2
15
page 6
20 3
20
12 2
12
3 0
3
2 0
2
1 0
1
7 1
7
6 1
6
13 2
13
11 2
11
5 0
5
page 7
13 2
13
11 2
11
8 1
8
12 2
12
2 0
2
10 1
10
1 0
1
11 2
11
2 0
2
5 0
5
page 8
6 1
6
5 0
5
7 1
7
3 0
3
6 1
6
8 1
8
15 2
15
11 2
11
1 0
1
3 0
3
page 9
2 0
2
16 3
16
1 0
1
16 3
16
1 0
1
9 1
9
1 0
1
5 0
5
7 1
7
2 0
2
page 10
7 1
7
1 0
1
1 0
1
3 0
3
4 0
4
1 0
1
8 1
8
21 4
21
26 5
26
9 1
9


In [42]:
len(answer_list)

3572

In [41]:
with open('gf_gift.txt', 'w', encoding ='utf-8') as f:
  for answer in answer_list:
    f.write(answer + '\n')

In [21]:
from urllib.parse import quote

base_url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&'
url = f"{base_url}query={quote('스마트폰')}"
driver.get(url)
time.sleep(1)

In [22]:
# 결과 화면에서 VIEW click
driver.find_element(By.XPATH, '//*[@id="lnb"]/div[1]/div/ul/li[6]/a').click()
time.sleep(3)

In [23]:
# Page Down 화살표를 사용하여 데이터를 충분히 가져오는 과정
body = driver.find_element(By.TAG_NAME, 'body')
for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(3)

In [24]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()

In [25]:
lis = soup.select('li.bx._svp_item')
len(lis)

104